In [ ]:
suppressPackageStartupMessages(library(dplyr)) 
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(harmony))

library(ggplot2)
library(pheatmap)
library(RColorBrewer)
library(repr)

source("/projects/ps-renlab2/y2xie/scripts/basics.R")
source("/projects/ps-renlab2/y2xie/scripts/DPT_help.R")

options(future.globals.maxSize = 64000 * 1024^2)

## Process single channel Multiome data

In [ ]:
### Process carehf object into one
library(foreach)
library(doParallel)
registerDoParallel(cores = 16)

allf <- list.files("../reference/CAREHF/", pattern = ".*.RmMult.rds")
crna_list <- foreach(f = allf, .packages = "Seurat") %dopar% {
    read_attempt <- try(readRDS(paste0("../reference/CAREHF/", f)), silent = TRUE)
    if (inherits(read_attempt, "try-error")) {
        stop("Error reading ", f, "- skipping to next file.\n")
    }
    mmg <- readRDS(paste0("../reference/CAREHF/", f))
    tmp <- mmg@meta.data

    rownames(tmp) <- paste0(tmp$samples, ":", rownames(tmp)) 
    
    mtx <- mmg[["RNA"]]@counts
    colnames(mtx) <- paste0(mmg$samples, ":", colnames(mtx))
    return(list(mtx, tmp))
}

### Process CAREHF DNA count matrix
dna_list <- foreach(f = allf, .packages = "Seurat") %dopar% {
    read_attempt <- try(readRDS(paste0("../reference/CAREHF/", f)), silent = TRUE)
    if (inherits(read_attempt, "try-error")) {
        stop("Error reading ", f, "- skipping to next file.\n")
    }
    mmg <- readRDS(paste0("../reference/CAREHF/", f))
    tmp <- mmg@meta.data

    rownames(tmp) <- paste0(tmp$samples, ":", rownames(tmp)) 
    
    mtx <- mmg[["ATAC"]]@counts
    colnames(mtx) <- paste0(mmg$samples, ":", colnames(mtx))
    return(list(mtx, tmp))
}

In [ ]:
mtx_list <- lapply(crna_list, function(x) x[[1]])
meta_list <- lapply(crna_list, function(x) x[[2]])

### merge object
tmp <- Reduce(cbind, mtx_list[-1], mtx_list[[1]]) ### dont use do.call for cbind
tmeta <- do.call(rbind, meta_list)
mmg <- CreateSeuratObject(tmp, meta.data = tmeta)

In [ ]:
ref2 <- subset(mmg, subset = is_cell == 1)
carehf_donor <- read.table("../reference/CARE_HF_sample.info", header = T, sep = "\t")
ref2@meta.data[,c('Chamber', 'Donor')] <- carehf_donor[match(ref2$samples, carehf_donor$ID), c('Chamber', 'Donor')] 

In [43]:
ref2 <- RunRNA2(ref2, batch.label = "samples")
qs::qsave(ref2, "../reference/CAREHF/merge.rna.qs", preset = "balanced", nthreads = 32)
ref2

An object of class Seurat 
69963 features across 252832 samples within 2 assays 
Active assay: SCT (33362 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: RNA
 3 dimensional reductions calculated: pca, harmony, umap